#### Imports

In [1]:
import pandas as pd
import re, nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import json
from pathlib import Path
import pandas as pd
import numpy as np

#### Constants

In [2]:
PATH = 'C:\\Users\\vinic\\test_directory\\tcc\\training.1600000.processed.noemoticon.csv'
COLUMNS = ["target", "ids", "date", "flag", "user", "text"]

#### Reading Sentiment140 dataset 1.6kk tweets 

In [3]:
df = pd.read_csv(PATH, encoding="ISO-8859-1", names=COLUMNS)
df_copy = df
df_copy = df_copy.drop(['ids', 'date', 'flag', 'user'], axis=1)

In [4]:
df_copy.head(10)

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew
6,0,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,@Tatiana_K nope they didn't have it
9,0,@twittera que me muera ?


#### Counting number of tweets and number of targets

In [5]:
#sentiment_counts = df_copy.target.value_counts()
#number_of_tweets = df_copy.ids.count()



#### Total number of tweets

In [6]:
len(df_copy)

1600000

### Pre-processing

* ##### Stopwords and Lemmatizer

In [7]:
wordnet_lemmatizer = WordNetLemmatizer()

* #### Preprocess function

In [8]:
import string
punct = list(string.punctuation)
stopword_list = punct + stopwords.words('english') + ['rt', 'via', '...', 'http', 'twitpic', 'tinyurl' ,'www']

[\VERSION 1.1]

In [9]:
from textblob import TextBlob

def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

# Lemmatize
#sentence = "The striped bats are hanging on their feet for best"
#> 'The striped bat be hang on their foot for best'

In [10]:
#version 1.0
def normalizer(tweet):
    punct = list(string.punctuation)
    stop_words = stopwords.words('english')
    additional_stop_words = ['rt', 'via', '...', 'http', 'twitpic', 'tinyurl' ,'www']
    stopword_list = punct + stop_words + additional_stop_words
    
    tweet = re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet) #removing hashtags and mentions
    tweet_ = re.sub("(\w+:\/\/\S+)", " ", tweet)
    tweet__ = re.sub("[^a-zA-Z]", " ", tweet_)
    lemmatized = lemmatize_with_postag(tweet__)
    tokens = nltk.word_tokenize(lemmatized)[2:]
    lower_case = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stopword_list, lower_case))
    #lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
    return filtered_result

#### NOT DEPRESSIVE Tweets
* ##### This tweets are from sentiment140 

In [11]:
not_depressive_tweets_df = df_copy[df_copy.target == 4]
not_depressive_tweets_df = not_depressive_tweets_df.sample(n=3574, random_state=27)

In [12]:
print(len(not_depressive_tweets_df))

3574


#### DEPRESSIVE Tweets
* ##### Converting .txt to JSON and importing DEPRESSIVE TWEETS in a DataFrame

In [13]:
file_to_open = Path("C:/Users/vinic/test_directory/tcc/Depression-Sentiment-Analysis-Twitter-Data-master", "tweetdata.txt")
with open(file_to_open) as datafile:
    content = datafile.read()
depressive_tweets_df = pd.read_json('[' + content.replace('}\n', '},') + ']')


* ##### Creating column with target 0, which the label for DEPRESSIVE tweets

In [14]:
target = pd.DataFrame(np.zeros(depressive_tweets_df.shape[0], dtype=int))
depressive_tweets_df['target'] = target
depressive_tweets_df[['target', 'text']].tail() #showing final DataFrame

,target,text
3569,0,I never get this much anxiety until I'm home :/
3570,0,Hospital staff warn of more 'adverse outcomes'...
3571,0,RT @Jniewalker: มินย้อกถามซอฮยอนว่า depression...
3572,0,RT @philuhmena: 7 months ago today I was raped...
3573,0,my anxiety is thru the fucking roof.


* ##### Checking if the number of samples from each classes are equal on the dataset. They need ot be balanced in order for the algorithm learns how to identify both classes without bias.

In [15]:
len_not_depress = len(not_depressive_tweets_df)
len_depress = len(depressive_tweets_df)
def equal(a,b):
    if (int(a)==int(b)):
        return print('> Nice! The number of classes are equal on dataset.')
    else:
        return print('> Number of classes aren\'t equal on dataset. :(')

equal(len_depress, len_not_depress)

> Nice! The number of classes are equal on dataset.


* #### Concatenating DEPRESSIVE and NOT DEPRESSIVE tweets

In [16]:
all_tweets = pd.concat([not_depressive_tweets_df, depressive_tweets_df], sort=False)

In [17]:
len(all_tweets)

7148

In [18]:
all_tweets['target'].value_counts()

4    3574
0    3574
Name: target, dtype: int64

In [19]:
all_tweets = all_tweets.sample(frac=1).reset_index(drop=True)

#### Verifying the number of labels by class

In [20]:
all_tweets.target.value_counts()

4    3574
0    3574
Name: target, dtype: int64

In [21]:
all_tweets.head()

,target,text,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,timestamp_ms,truncated,user
0,4,Bond fireeeee,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,4,@Hann_Oxx yep ty omg thank god schools over :L x,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,4,Found it! Thanks everyone,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,0,@MariaBurnham4 wow! Well done! Having a little...,NaN,None,2017-06-02 00:20:43,"[15, 126]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,...,NaN,NaN,NaN,0.0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",2017-06-02 00:20:43.320,False,"{'id': 405869972, 'id_str': '405869972', 'name..."
4,4,Lunch with the &quot;fam&quot;,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN


In [22]:
all_tweets['normalized_tweet'] = all_tweets.text.apply(normalizer)

In [23]:
all_tweets.head()

,target,text,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,timestamp_ms,truncated,user,normalized_tweet
0,4,Bond fireeeee,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,[]
1,4,@Hann_Oxx yep ty omg thank god schools over :L x,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,"[ty, omg, thank, god, school, l, x]"
2,4,Found it! Thanks everyone,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,"[thanks, everyone]"
3,0,@MariaBurnham4 wow! Well done! Having a little...,NaN,None,2017-06-02 00:20:43,"[15, 126]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,...,NaN,NaN,0.0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",2017-06-02 00:20:43.320,False,"{'id': 405869972, 'id_str': '405869972', 'name...","[little, anxiety, prepare, deployment, grade, ..."
4,4,Lunch with the &quot;fam&quot;,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,"[quot, fam, quot]"


#### Creating vector of bigrams and trigrams

In [24]:
all_tweets.columns

Index(['target', 'text', 'contributors', 'coordinates', 'created_at',
       'display_text_range', 'entities', 'extended_entities', 'extended_tweet',
       'favorite_count', 'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quoted_status', 'quoted_status_id',
       'quoted_status_id_str', 'retweet_count', 'retweeted',
       'retweeted_status', 'source', 'timestamp_ms', 'truncated', 'user',
       'normalized_tweet'],
      dtype='object')

In [25]:
from nltk import ngrams
def ngrams(input_list):
    #onegrams = input_list
    bigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:]))]
    #trigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:], input_list[2:]))]
    return bigrams

all_tweets['grams'] = all_tweets.normalized_tweet.apply(ngrams)
all_tweets['grams'].head()

0                                                   []
1    [ty omg, omg thank, thank god, god school, sch...
2                                    [thanks everyone]
3    [little anxiety, anxiety prepare, prepare depl...
4                                 [quot fam, fam quot]
Name: grams, dtype: object

#### Now some counting

In [26]:
import collections
def count_words(input):
    cnt = collections.Counter()
    for row in input:
        for word in row:
            cnt[word] += 1
    return cnt

### Looking at the 20 most common n-grams

#### DEPRESSIVE Tweets

In [27]:
all_tweets[(all_tweets.target == 0)][['grams']].apply(count_words)['grams'].most_common(20)

[('mental health', 558),
 ('resting bitch', 208),
 ('bitch face', 208),
 ('want friend', 207),
 ('friend w', 207),
 ('w ppl', 207),
 ('ppl ppl', 207),
 ('ppl think', 207),
 ('think ur', 207),
 ('ur mean', 207),
 ('mean unapproachable', 207),
 ('unapproachable cu', 207),
 ('cu u', 207),
 ('u resting', 207),
 ('face soci', 207),
 ('go miss', 186),
 ('think go', 185),
 ('miss eighth', 185),
 ('eighth grade', 185),
 ('grade tough', 185)]

#### NOT DEPRESSIVE Tweets

In [28]:
all_tweets[(all_tweets.target == 4)][['grams']].apply(count_words)['grams'].most_common(20)

[('gon na', 36),
 ('last night', 19),
 ('wan na', 14),
 ('look like', 14),
 ('let know', 13),
 ('good luck', 12),
 ('wait see', 12),
 ('good night', 11),
 ('day use', 10),
 ('much good', 10),
 ('com add', 9),
 ('add everyone', 9),
 ('everyone train', 9),
 ('train pay', 9),
 ('pay vip', 9),
 ('cant wait', 9),
 ('good day', 9),
 ('first time', 8),
 ('feel well', 8),
 ('new song', 8)]

#### Instantiating CountVectorizer

In [29]:
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1,2))

In [30]:
vectorized_data = count_vectorizer.fit_transform(all_tweets.text)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))

In [31]:
count_vectorizer.vocabulary_

{'bond': 7842,
 'fireeeee': 18221,
 'bond fireeeee': 7843,
 'hann_oxx': 22175,
 'yep': 59926,
 'ty': 55071,
 'omg': 37847,
 'thank': 50127,
 'god': 20679,
 'schools': 44994,
 'over': 38960,
 'hann_oxx yep': 22176,
 'yep ty': 59937,
 'ty omg': 55074,
 'omg thank': 37863,
 'thank god': 50132,
 'god schools': 20697,
 'schools over': 45002,
 'found': 19108,
 'it': 26785,
 'thanks': 50152,
 'everyone': 16995,
 'found it': 19114,
 'it thanks': 27075,
 'thanks everyone': 50170,
 'mariaburnham4': 32129,
 'wow': 59263,
 'well': 57414,
 'done': 15315,
 'having': 22745,
 'little': 30527,
 'anxiety': 4101,
 'here': 23545,
 'preparing': 40883,
 'for': 18653,
 'our': 38744,
 'deployment': 14058,
 'with': 58473,
 'grades': 21273,
 'in': 25396,
 'the': 50602,
 'fall': 17464,
 'mariaburnham4 wow': 32130,
 'wow well': 59290,
 'well done': 57430,
 'done having': 15323,
 'having little': 22763,
 'little anxiety': 30528,
 'anxiety over': 4296,
 'over here': 38976,
 'here preparing': 23589,
 'preparing for'

In [32]:
targets = all_tweets.target

#### Train Test Split

In [33]:
from sklearn.model_selection import train_test_split
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, targets, test_size=0.3, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

In [34]:
from sklearn import svm

clf = svm.SVC(C=1, kernel='linear')
clf_output = clf.fit(data_train, targets_train)

#### Model Score

In [35]:
clf_output.score(data_test, targets_test)

0.9948717948717949

In [36]:
len(targets)

7148

#### GRID-SEARCH

In [42]:
from sklearn.model_selection import GridSearchCV
hiperparameters = {'kernel': ('linear', 'rbf','poly'), 
              'C':[1.5, 10, 100, 1000],
              'gamma': [1e-7, 1e-4, 1e-2, 0.1]}
grid = GridSearchCV(svm.SVC(), hiperparameters, cv=5)
grid.fit(data_train, targets_train)

print(grid.best_params_)
print(grid.best_score_)

{'C': 1.5, 'gamma': 0.01, 'kernel': 'rbf'}
0.9974015590645613


In [ ]:
"""
from sklearn.model_selection import GridSearchCV

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

# add this kernels polynominal, sigmoid

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(data_train, targets_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    print("Detailed classification report:")
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    y_true, y_pred = targets_test, clf.predict(data_test)
    print(classification_report(y_true, y_pred))
"""


#### Predicting

In [ ]:
tweet_to_predict = ['I have so many reasons to smile']


In [ ]:
#txt = normalizer(tweet_to_predict)
#text = count_vectorizer.fit_transform(normalizer(tweet_to_predict))
#vectorized_data = count_vectorizer.fit_transform(all_tweets.text)



In [ ]:
#text = count_vectorizer.transform(normalizer(tweet_to_predict))
#unseen = (np.array(range(0,vectorized_data.shape[0])))


In [ ]:
#clf_output.predict(unseen)